In [1]:
# | default_exp llms.response_processor

In [2]:
# | export 
import json
from typing import Dict, Any, Optional
from rich.console import Console
from rich.markdown import Markdown
from agentic.configs.loader import get_reasoning_config
import re

In [3]:
# | export
RESET = "\033[0m"
# Get reasoning config
reasoning_config = get_reasoning_config()
show_thinking = reasoning_config.get('show_thinking', True)
thinking_color = reasoning_config.get('thinking_color', 'pink')

# Set color based on config with enhanced colors
color_codes = {
            'pink': "\033[38;2;200;100;120m",     # Darker pink
            'blue': "\033[38;2;70;130;180m",      # Steel blue
            'green': "\033[38;2;60;179;113m",     # Medium sea green
            'yellow': "\033[38;2;204;204;0m",     # Olive-like dark yellow
            'purple': "\033[38;2;147;112;219m",   # Medium purple
            'cyan': "\033[38;2;0;139;139m"        # Dark cyan
        }


color_code = color_codes.get(thinking_color.lower(), color_codes['green'])   
def show_thinking_header():
    """Display beautiful thinking header"""
    if show_thinking:
        print(f"\n{color_code}╭─────────────────────── 🤔 Thinking ───────────────────────╮{RESET}")

def show_thinking_footer():
    """Display beautiful thinking footer"""
    if show_thinking:
        print(f"{color_code}╰────────────────────────────────────────────────────────────╯{RESET}")

In [4]:
# | export
class ResponseProcessor:
    """Handles processing of LLM responses"""
    
    def __init__(self):
        self.console = Console()
    
    def process_response(self, response: Any, console: Optional[Console] = None) -> Dict[str, Any]:
        """Process non-streaming response"""
        if console is None:
            console = self.console
        
        try:
            message = response.choices[0].message
            
            cleaned_content = ""
            if hasattr(message, 'reasoning') and message.reasoning:
                show_thinking_header()
                console.print(Markdown(message.reasoning))
                show_thinking_footer()
            if hasattr(message, 'content') and message.content:
                content = message.content
        
                # Extract <think>...</think> content
                think_matches = re.findall(r"<think>(.*?)</think>", content, re.DOTALL)
        
                # Remove <think>...</think> blocks from original content
                cleaned_content = re.sub(r"<think>.*?</think>", "", content, flags=re.DOTALL)
    
                # Print each <think> block inside the fancy box
                for think in think_matches:
                    think = think.strip()
                    if think:
                        show_thinking_header()
                        print(f"{color_code}{think}{RESET}")
                        show_thinking_footer()
        
                # Print main message content
                if cleaned_content.strip():
                    console.print(Markdown(cleaned_content.strip()))
            
            # Extract tool calls
            tool_calls = []
            if hasattr(message, 'tool_calls') and message.tool_calls:
                tool_calls = self._extract_tool_calls(message.tool_calls)
            
            return {
                "content": cleaned_content,
                "tool_calls": tool_calls,
                "finish_reason":response.choices[0].finish_reason,
                "usage": getattr(response, 'usage', None),
                "model": getattr(response, 'model', None)
            }
            
        except Exception as e:
            console.print(f"[red]Error processing response: {e}[/red]")
            return {"content": "", "tool_calls": [], "error": str(e)}
    
    def _extract_tool_calls(self, tool_calls) -> list:
        """Extract tool calls from response"""
        extracted_calls = []
        
        for tool_call in tool_calls:
            try:
                extracted_call = {
                    "id": tool_call.id,
                    "type": tool_call.type,
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    }
                }
                extracted_calls.append(extracted_call)
            except Exception as e:
                print(f"Error extracting tool call: {e}")
                continue
        
        return extracted_calls


In [5]:
# Example usage 
from agentic.llms.client import LLMClient
llm_client = LLMClient(model="qwen3:14b",base_url="http://192.168.29.147:11500/v1",api_key="ollama")
response = llm_client.create_completion(
            messages=[{"role":"user","content":"Why is sky blue?"}], stream=False
        )
result = llm_client.process_response(response)


╭─────────────────────── 🤔 Thinking ───────────────────────╮
Okay, the user is asking why the sky is blue. I need to explain the science behind it. Let me start by recalling what I know about the scattering of light in the atmosphere.

First, sunlight appears white, but it's made up of different colors, each with different wavelengths. Blue light has a shorter wavelength compared to colors like red or yellow. When sunlight enters Earth's atmosphere, it collides with molecules and small particles in the air. This interaction causes the light to scatter in all directions.

I remember that Rayleigh scattering is the main principle here. It states that the intensity of scattered light is inversely proportional to the fourth power of the wavelength. So shorter wavelengths (blue, violet) scatter more than longer ones (red, orange). That's why the sky appears blue instead of violet, even though violet has an even shorter wavelength. Wait, why isn't it violet then?

Oh right, the sun emits mo

The sky appears blue due to a phenomenon called Rayleigh scattering, which involves the interaction of sunlight    
with molecules and small particles in Earth's atmosphere. Here's a step-by-step explanation:                       

 1 Sunlight Composition:                                                                                           
   Sunlight consists of a spectrum of colors (wavelengths), from violet (shortest) to red (longest). While sunlight
   appears white, it contains more blue light than violet or other colors.                                         
 2 Scattering in the Atmosphere:                                                                                   
   When sunlight enters Earth's atmosphere, it collides with molecules (like nitrogen and oxygen) and tiny         
   particles. This causes the light to scatter in all directions.                                                  
 3 Rayleigh Scattering:                                                                                            
   Rayleigh scattering occurs when light interacts with particles much smaller than its wavelength (like           
   atmospheric molecules). The key rule here is that shorter wavelengths (blue/violet) scatter much more strongly  
   than longer wavelengths (red/orange). Specifically, the intensity of scattered light is inversely proportional  
   to the fourth power of the wavelength—so blue light (with a shorter wavelength) scatters about 9 times more than
   red light.                                                                                                      
 4 Why Blue, Not Violet?:                                                                                          
   Although violet light scatters even more than blue, the sky isn’t violet for two reasons:                       
    • The sun emits more blue light than violet.                                                                   
    • Human eyes are more sensitive to blue.                                                                       
    • Some violet light is absorbed by ozone in the atmosphere.                                                    
 5 Daytime vs. Sunset Colors:                                                                                      
    • During the day: Blue light is scattered in all directions, making the sky appear blue from all angles.       
    • At sunrise/sunset: Light travels through more atmosphere, scattering out most of the blue and leaving longer 
      wavelengths (red/orange) to dominate the sky.                                                                
 6 Other Influences:                                                                                               
    • Pollutants, dust, and larger particles can scatter light differently, sometimes making the sky appear white  
      or hazy.                                                                                                     
    • Over water, Rayleigh scattering also contributes to the ocean’s blue color, as water absorbs longer          
      wavelengths and reflects scattered blue light.                                                               

In short, the sky is blue because Earth’s atmosphere scatters shorter wavelengths of sunlight (blue) more          
effectively than longer ones, and our eyes perceive this scattered blue light as the color of the sky. 🌤️

In [13]:
result.usage

CompletionUsage(completion_tokens=911, prompt_tokens=13, total_tokens=924, completion_tokens_details=None, prompt_tokens_details=None)

In [11]:
from pprint import pprint
pprint(result.model_dump())

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': None,
                          'audio': None,
                          'content': '<think>\n'
                                     'Okay, the user is asking why the sky is '
                                     'blue. I need to explain the science '
                                     'behind it. Let me start by recalling '
                                     'what I know about the scattering of '
                                     'light in the atmosphere.\n'
                                     '\n'
                                     "First, sunlight appears white, but it's "
                                     'made up of different colors, each with '
                                     'different wavelengths. Blue light has a '
                                     'shorter wavelength compared to colors '
                                